In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

arms = 10
round = 5000
simulate = 1000
means = np.linspace(0.1,0.9, arms) 

def get_reward(mean):
    n = np.random.rand()
    if n < mean:
        return 1
    else:
        return 0

def cumulative_regret(regret):
    cumu_regret = np.zeros_like(regret)
    cumu_regret[0] = regret[0]
    for i in range(1,round):
        cumu_regret[i] = regret[i] + cumu_regret[i-1]
    return cumu_regret

def uniform_sampling(round,means,simulate):
    arms = len(means)
    regret_matrix = np.zeros((simulate,round))
    optimal_mean = np.max(means)

    for t in range(simulate):
        rewards = np.zeros(arms)
        arm_play = np.zeros(arms)
        regret = np.zeros(round)

        for i in range(round):
            arm = np.random.randint(arms)
            reward = get_reward(means[arm])
            arm_play[arm] += 1
            rewards[arm] += reward
            regret[i] = (optimal_mean - means[arm])

        regret_matrix[t] = cumulative_regret(regret)

    avg_regret = np.mean(regret_matrix, axis=0)
    std_regret = np.std(regret_matrix, axis=0)
    return avg_regret, std_regret

def Greedy(round,means,simulate,epsilon):
    arms = len(means)
    regret_matrix = np.zeros((simulate,round))
    optimal_mean = np.max(means)

    for t in range(simulate):
        rewards = np.zeros(arms)
        arm_play = np.zeros(arms)
        regret = np.zeros(round)
        exp_reward = np.zeros(arms)
        for i in range(round):

            l = np.random.rand()
            if l<epsilon:
                arm = np.random.randint(arms)
            else:
                for j in range(arms):
                    if arm_play[j] != 0:
                        exp_reward[j] = rewards[j]/arm_play[j]
                arm = np.argmax(exp_reward)

            #arm = np.random.randint(arms)
            reward = get_reward(means[arm])
            arm_play[arm] += 1
            rewards[arm] += reward
            regret[i] = (optimal_mean - means[arm])

        regret_matrix[t] = cumulative_regret(regret)

    avg_regret = np.mean(regret_matrix, axis=0)
    std_regret = np.std(regret_matrix, axis=0)
    return avg_regret, std_regret

def UCB(round,means,simulate):
    arms = len(means)
    regret_matrix = np.zeros((simulate,round))
    optimal_mean = np.max(means)

    for t in range(simulate):
        rewards = np.zeros(arms)
        arm_play = np.zeros(arms)
        regret = np.zeros(round)
        exp_reward = np.zeros(arms)
        
        for i in range(round):
            if i < arms:
                arm = i 
            else:
                for j in range(arms):
                    a = math.sqrt((2 * math.log(i)) / arm_play[j] )
                    exp_reward[j] = rewards[j]/arm_play[j] + a

                arm = np.argmax(exp_reward)

            reward = get_reward(means[arm])
            arm_play[arm] += 1
            rewards[arm] += reward
            regret[i] = (optimal_mean - means[arm])

        regret_matrix[t] = cumulative_regret(regret)

    avg_regret = np.mean(regret_matrix, axis=0)
    std_regret = np.std(regret_matrix, axis=0)
    return avg_regret, std_regret

def thompson(round,means,simulate):
    arms = len(means)
    regret_matrix = np.zeros((simulate,round))
    optimal_mean = np.max(means)

    for t in range(simulate):
        rewards = np.zeros(arms)
        arm_play = np.zeros(arms)
        regret = np.zeros(round)
        alpha = np.ones(arms)
        beta = np.ones(arms)
        
        for i in range(round):
            theta_samples = np.random.beta(alpha, beta)
            arm = np.argmax(theta_samples)
            
            reward = get_reward(means[arm])
            if reward == 1:
                alpha[arm] += 1
            else:
                beta[arm] += 1

            arm_play[arm] += 1
            rewards[arm] += reward
            regret[i]  = (optimal_mean - means[arm])
        
        regret_matrix[t] = cumulative_regret(regret)

    avg_regret = np.mean(regret_matrix, axis=0)
    std_regret = np.std(regret_matrix, axis=0)
    return avg_regret, std_regret


regret_1, std_1 = uniform_sampling(round, means, simulate)
regret_2, std_2 = Greedy(round, means, simulate, epsilon=0.1)
regret_3, std_3 = UCB(round, means, simulate)
regret_4, std_4 = thompson(round, means, simulate)



plt.plot(regret_1, label='ε-Greedy (ε=1)')
plt.fill_between(range(round), regret_1 - std_1, regret_1 + std_1,alpha=0.2)
plt.plot(regret_2, label='ε-Greedy (ε=0.1)',)
plt.fill_between(range(round), regret_2 - std_2, regret_2 + std_2,alpha=0.2)
plt.plot(regret_3, label='UCB', color='red')
plt.fill_between(range(round), regret_3 - std_3, regret_3 + std_3,alpha=0.2)
plt.plot(regret_4, label='Thompson Sampling',)
plt.fill_between(range(round), regret_4 - std_4, regret_4 + std_4,alpha=0.2)
plt.xlabel('Rounds')
plt.ylabel('Cumulative Regret')
plt.legend()
plt.title('Average Cumulative Regret vs. Rounds for Different Algorithms with Standard Deviation')
plt.show()